## Import Libraries

In [1]:
import pandas as pd
import numpy as np

## Load csv data and merge them to create a standard file

#### Step 1: JD Power Data

In [ ]:
# STEP 1
# Load rating data from JD Power source
df_rating_JD = pd.read_csv("C:/Users/MyPC/Documents/GitHub/Carma/db/data/cleaned_data/jdpower_cleaned_really_final.csv")

# Drop the first column "Unnamed" 
del df_rating_JD[df_rating_JD.columns[0]]
df_rating_JD.head()

#### Step 2: KBB Customer Reviews

In [ ]:
# Load rating data from KBB customer reviews
df_rating_KBB_customers = pd.read_csv("C:/Users/MyPC/Documents/GitHub/Carma/db/data/cleaned_data/KBB_consumer_ratings_standard.csv")

# Upper case all the makes and models
df_rating_KBB_customers['Make'] = df_rating_KBB_customers['Make'].str.upper()
df_rating_KBB_customers['Model'] = df_rating_KBB_customers['Model'].str.upper()

df_rating_KBB_customers.head()

#### Step 3: KBB mpg 

In [ ]:
# Load rating data from KBB mpg standard
df_rating_KBB_mpg = pd.read_csv("C:/Users/MyPC/Documents/GitHub/Carma/db/data/cleaned_data/KBB_mpg_standard.csv")

# Upper case all the makes and models
df_rating_KBB_mpg['Make'] = df_rating_KBB_mpg['Make'].str.upper()
df_rating_KBB_mpg['Model'] = df_rating_KBB_mpg['Model'].str.upper()

df_rating_KBB_mpg.head()

#### Step 4: KBB ratings 

In [ ]:
# Load rating data from KBB_ratings_standard
df_rating_KBB = pd.read_csv("C:/Users/MyPC/Documents/GitHub/Carma/db/data/cleaned_data/KBB_ratings_standard.csv")

# Upper case all the makes and models
df_rating_KBB['Make'] = df_rating_KBB['Make'].str.upper()
df_rating_KBB['Model'] = df_rating_KBB['Model'].str.upper()

df_rating_KBB.head()

#### Step 5: Standard List

In [ ]:
df_standards = pd.read_csv("C:/Users/MyPC/Documents/GitHub/Carma/db/data/THE STANDARD.csv")

# Upper case all the makes and models
df_standards['make'] = df_standards['make'].str.upper()
df_standards['model'] = df_standards['model'].str.upper()

standard_make = list(df_standards['make'])
standard_model = list(df_standards['model'])

#Rename df_standards to outer join with df_genres later
df_standards.set_axis(['Year','Make','Model','Body Styles'], axis=1, inplace=True)
df_standards.head()

In [ ]:
# Remove rows that have invalid entries ("make","model","year","body styles") from the standards

print('before remove invalid values', len(df_standards))
# Get names of indexes for which column Age has value ''
indexNames = df_standards[df_standards['Year'] == 'year'].index
# Delete these row indexes from dataFrame
df_standards.drop(indexNames , inplace=True)

print('after remove null values', len(df_standards))
df_standards.head()

In [ ]:
#convert list to dictionary to index efficiently for make
standard_make_dict_l = {}
standard_make_dict = {}
for i in range(len(standard_make)):
    standard_make_dict_l[standard_make[i].lower()] = i
    standard_make_dict[standard_make[i]] = i
#standard_make_dict

In [ ]:
#convert list to dictionary to index efficiently for model
standard_model_dict_l = {}
standard_model_dict = {}
for i in range(len(standard_model)):
    standard_model_dict_l[standard_model[i].lower()] = i
    standard_model_dict[standard_model[i]] = i
#standard_model_dict

#### Step 6: Recall data

In [ ]:
# Load rating data from recalls
df_rating_recalls = pd.read_csv("C:/Users/MyPC/Documents/GitHub/Carma/db/data/NHTSA_recalls.csv")
df_rating_recalls.head(10)

In [ ]:
len(df_rating_recalls)

In [ ]:
# Transform data to keep max severity score for each make, model, year.
df_rating_recalls_max = df_rating_recalls.groupby(['MAKETXT','MODELTXT','YEARTXT'], sort=False)['SEVERITY_SCORE'].max().reset_index()

In [ ]:
# Compare results before and after max transformation
print('Original Dataset')
print(df_rating_recalls.groupby(['MAKETXT', 'MODELTXT','YEARTXT']).ngroups)
print(len(df_rating_recalls))

print('Max Dataset')
print(df_rating_recalls_max.groupby(['MAKETXT', 'MODELTXT','YEARTXT']).ngroups)
print(len(df_rating_recalls_max))

In [ ]:
# Standardize recalls data
def standardize_make_recalls(row):
    try:
        result = standard_make[standard_make_dict_l[row['MAKETXT'].lower()]]
    except:
        result = ''
    return result

def standardize_model_recalls(row):
    try:
        result = standard_model[standard_model_dict_l[row['MODELTXT'].lower()]]
    except:
        result = ''
    return result

df_rating_recalls_max['Make'] = df_rating_recalls_max.apply(standardize_make_recalls, axis = 1)
df_rating_recalls_max['Model'] = df_rating_recalls_max.apply(standardize_model_recalls, axis = 1)
df_rating_recalls_max.head()

In [ ]:
# Remove rows that do not have (make,model,year) from the standards

print('before remove null values', len(df_rating_recalls_max))
# Get names of indexes for which column Age has value ''
indexNames = df_rating_recalls_max[df_rating_recalls_max['Model'] == ''].index
# Delete these row indexes from dataFrame
df_rating_recalls_max.drop(indexNames , inplace=True)

print('after remove null values', len(df_rating_recalls_max))
df_rating_recalls_max.head()

In [ ]:
#Clean the data
#Drop columns 'MAKETXT', 'MODELTXT' and rename 'YEARTXT'
df_rating_recalls_max = df_rating_recalls_max.drop(['MAKETXT','MODELTXT'], axis = 1)

# Rename columns:
df_rating_recalls_max = df_rating_recalls_max.rename(columns={"YEARTXT": "Year", "SEVERITY_SCORE": "Recall Score"})

df_rating_recalls_max.head()

In [ ]:
# Save file to disk
df_rating_recalls_max.to_csv('recalls_standard.csv', index=False)

##### Step 7: KBB micro genres

In [ ]:
# Load rating data from KBB_micro_genres
df_KBB_genres = pd.read_csv("C:/Users/MyPC/Documents/GitHub/Carma/db/data/cleaned_data/car_micro_genres.csv")

# Upper case all the makes and models
df_KBB_genres['make'] = df_KBB_genres['make'].str.upper()
df_KBB_genres['model'] = df_KBB_genres['model'].str.upper()

df_KBB_genres.set_axis(['Make','Model','Off_road','Family','Fun','Luxury'], axis=1, inplace=True)
df_KBB_genres.head()

In [ ]:
# dropping duplicate values 
print('before',len(df_KBB_genres))
df_KBB_genres_unique = df_KBB_genres.drop_duplicates(keep='first') 
print('after',len(df_KBB_genres_unique))

In [ ]:
df_KBB_genres_unique.head()

In [ ]:
# Note that now, we have 1239 genres but 9421 cars in the standards, we need to add those cars in the genres and set the genres
# to False, False, False, False

#Outer join df_standards withdf_genres and replace the NaN by False
print('df_standards: ', len(df_standards), 'vs df_KBB_genres_unique: ', len(df_KBB_genres_unique))
data_frames = [df_standards,df_KBB_genres_unique]

from functools import reduce
df_KBB_genres_update = reduce(lambda  left,right: pd.merge(left,right,on=['Make', 'Model'],
                                            how='outer'), data_frames)
df_KBB_genres_update.head()
print('df_KBB_genres_update', len(df_KBB_genres_update))

In [ ]:
int(df_KBB_genres_update['Year'][0]) 
#df_KBB_genres_update['Year'].astype(int)
df_KBB_genres_update['Year'] = pd.to_numeric(df_KBB_genres_update['Year'])

In [ ]:
df_KBB_genres_update.head(10)

In [ ]:
#df_KBB_genres_update
# Save file to disk
df_KBB_genres_update.to_csv('genres_updates.csv', index=False)

##### Step 8: Join car specifications/ statistics data

In [ ]:
# Merge 5 tables KBB customer rating, KBB mpg, KBB overall rating, recalls and JD Power data
from functools import reduce

data_frames = [df_rating_KBB_customers, df_rating_KBB_mpg, df_rating_KBB, df_rating_recalls_max, df_rating_JD]

df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['Make', 'Model', 'Year'],
                                            how='outer'), data_frames)
df_merged.tail(20)

In [ ]:
# Add micro genres data
from functools import reduce

data_frames = [df_merged, df_KBB_genres_unique]

df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['Make', 'Model'],
                                            how='left'), data_frames)
df_merged.head()

In [ ]:
df_merged.tail()

In [ ]:
# Rename the title
df_merged.set_axis(['Make','Model','Year','KBB Cons Rating','KBB Combined Fuel Economy','KBB Expert Rating','Recall Score','JD Overall Rating','JD Quality and Reliability','JD Driving Experience','JD Resale','Genre Off Road','Genre Family','Genre Fun','Genre Luxury'], axis=1, inplace=True)
df_merged.tail()

In [ ]:
# Convert KBB Cons Rating and KBB Expert Rating from string/nan to numeric (float)
df_merged['KBB Cons Rating'] = df_merged['KBB Cons Rating'].astype(float)
df_merged['KBB Expert Rating'] = df_merged['KBB Expert Rating'].astype(float)
df_merged.head()

In [ ]:
df_merged.tail()

In [ ]:
# Correction for KBB Combined Fuel Economy for electric car
# Done in excell to correct value '105' to '30' for Ford Focus 2012,2013,2014 and 2016

In [ ]:
# Create another column called 'Aggregate Rating' which takes the following priorities: 'KBB Expert Rating', 'JD Driving Experience', 'KBB Cons Rating'
def get_overall_rating(row):
    options = np.array([row['KBB Expert Rating']*10,row['JD Driving Experience'],row['KBB Cons Rating']*10])
    try:
        result = options[np.isfinite(options)][0]
    except:
        result = float('nan')
    return result

df_merged['Aggregate Rating'] = df_merged.apply(get_overall_rating, axis=1)
df_merged.head()

In [ ]:
# Correction of KBB Combined Fuel Economy:
def fuel_correction(row):
    try:
        target_index = row['KBB Combined Fuel Economy'].find('|')
        if target_index >= 0:
            return float(row['KBB Combined Fuel Economy'][(target_index+1):])
        #return True
        else:
            return float(row['KBB Combined Fuel Economy'])
    except:
        return row['KBB Combined Fuel Economy']


In [ ]:
df_merged['KBB Combined Fuel Economy'] = df_merged.apply(fuel_correction, axis=1)
df_merged.head(10)

In [ ]:
# Save file to disk
df_merged.to_csv('aggregated_cleaned_data_7.csv', index=False)

## Load csv car statistics data from the previous step and merge it to the csv listing data

##### Get Car Statistics Data

In [2]:
df_car_stats = pd.read_csv("aggregated_cleaned_data_7.csv")

In [3]:
df_car_stats.head()

,Make,Model,Year,KBB Cons Rating,Scaled KBB Cons Rating,KBB Combined Fuel Economy,KBB Expert Rating,Recall Score,JD Overall Rating,Reverted Recall Score,JD Quality and Reliability,Scaled JD Quality and Reliability,JD Driving Experience,JD Resale,Genre Off Road,Genre Family,Genre Fun,Genre Luxury,Aggregate Rating
0,ACURA,ILX,2020,4.4,85.0,28.0,NaN,NaN,78.0,10,77.0,73.239437,72.0,77.0,False,False,True,True,72.0
1,ACURA,ILX,2019,4.4,85.0,28.0,4.4,6.0,78.0,4,77.0,73.239437,72.0,77.0,False,False,True,True,44.0
2,ACURA,ILX,2018,4.4,85.0,29.0,4.3,6.0,79.0,4,83.0,81.690141,70.0,77.0,False,False,True,True,43.0
3,ACURA,ILX,2017,4.4,85.0,29.0,4.4,6.0,72.0,4,72.0,66.197183,62.0,77.0,False,False,True,True,44.0
4,ACURA,ILX,2016,4.4,85.0,29.0,4.4,8.0,76.0,2,78.0,74.647887,67.0,75.0,False,False,True,True,44.0


In [4]:
# Replace NaN value in recall with 0
df_car_stats["Recall Score"] = df_car_stats["Recall Score"].fillna(0)

# Revert score for recall
df_car_stats["Recall Score"] = 10 - df_car_stats["Recall Score"] 

df_car_stats.head(10)

,Make,Model,Year,KBB Cons Rating,Scaled KBB Cons Rating,KBB Combined Fuel Economy,KBB Expert Rating,Recall Score,JD Overall Rating,Reverted Recall Score,JD Quality and Reliability,Scaled JD Quality and Reliability,JD Driving Experience,JD Resale,Genre Off Road,Genre Family,Genre Fun,Genre Luxury,Aggregate Rating
0,ACURA,ILX,2020,4.4,85.0,28.0,NaN,10.0,78.0,10,77.0,73.239437,72.0,77.0,False,False,True,True,72.0
1,ACURA,ILX,2019,4.4,85.0,28.0,4.4,4.0,78.0,4,77.0,73.239437,72.0,77.0,False,False,True,True,44.0
2,ACURA,ILX,2018,4.4,85.0,29.0,4.3,4.0,79.0,4,83.0,81.690141,70.0,77.0,False,False,True,True,43.0
3,ACURA,ILX,2017,4.4,85.0,29.0,4.4,4.0,72.0,4,72.0,66.197183,62.0,77.0,False,False,True,True,44.0
4,ACURA,ILX,2016,4.4,85.0,29.0,4.4,2.0,76.0,2,78.0,74.647887,67.0,75.0,False,False,True,True,44.0
5,ACURA,ILX,2015,4.4,85.0,25.0,3.4,2.0,74.0,2,80.0,77.464789,61.0,74.0,False,False,True,True,34.0
6,ACURA,ILX,2014,4.4,85.0,28.0,3.5,2.0,77.0,2,82.0,80.281690,56.0,73.0,False,False,True,True,35.0
7,ACURA,ILX,2013,4.4,85.0,28.0,3.5,2.0,81.0,2,87.0,87.323944,63.0,72.0,False,False,True,True,35.0
8,ACURA,RLX,2019,4.3,82.5,23.0,4.0,4.0,NaN,4,NaN,-35.211268,NaN,NaN,False,False,True,True,40.0
9,ACURA,RLX,2018,4.3,82.5,23.0,4.0,10.0,NaN,10,NaN,-35.211268,NaN,NaN,False,False,True,True,40.0


In [5]:
# Scale variables to scale of 100: (value - min)/(max-min)*100
# Variables to scale include: "JD Quality and Reliability", "Recall Score", "KBB Cons Rating", "JD Driving Experience", 
# "Aggregate Rating", "JD Resale", "KBB Combined Fuel Economy"

scaled_variable_names = ["JD Quality and Reliability", "Recall Score", "KBB Cons Rating", "JD Driving Experience", "Aggregate Rating", "JD Resale", "KBB Combined Fuel Economy"]
for name in scaled_variable_names:
    
    min_value = df_car_stats[name].min()
    max_value = df_car_stats[name].max()
    print(name, 'has min of', min_value, 'and max of', max_value)

    df_car_stats[name] = (df_car_stats[name]-min_value)/(max_value - min_value)*100

df_car_stats.head(10)

JD Quality and Reliability has min of 25.0 and max of 96.0
Recall Score has min of 0.0 and max of 10.0
KBB Cons Rating has min of 1.0 and max of 5.0
JD Driving Experience has min of 17.0 and max of 95.0
Aggregate Rating has min of 15.0 and max of 94.0
JD Resale has min of 49.0 and max of 95.0
KBB Combined Fuel Economy has min of 10.0 and max of 136.0


,Make,Model,Year,KBB Cons Rating,Scaled KBB Cons Rating,KBB Combined Fuel Economy,KBB Expert Rating,Recall Score,JD Overall Rating,Reverted Recall Score,JD Quality and Reliability,Scaled JD Quality and Reliability,JD Driving Experience,JD Resale,Genre Off Road,Genre Family,Genre Fun,Genre Luxury,Aggregate Rating
0,ACURA,ILX,2020,85.0,85.0,14.285714,NaN,100.0,78.0,10,73.239437,73.239437,70.512821,60.869565,False,False,True,True,72.151899
1,ACURA,ILX,2019,85.0,85.0,14.285714,4.4,40.0,78.0,4,73.239437,73.239437,70.512821,60.869565,False,False,True,True,36.708861
2,ACURA,ILX,2018,85.0,85.0,15.079365,4.3,40.0,79.0,4,81.690141,81.690141,67.948718,60.869565,False,False,True,True,35.443038
3,ACURA,ILX,2017,85.0,85.0,15.079365,4.4,40.0,72.0,4,66.197183,66.197183,57.692308,60.869565,False,False,True,True,36.708861
4,ACURA,ILX,2016,85.0,85.0,15.079365,4.4,20.0,76.0,2,74.647887,74.647887,64.102564,56.521739,False,False,True,True,36.708861
5,ACURA,ILX,2015,85.0,85.0,11.904762,3.4,20.0,74.0,2,77.464789,77.464789,56.410256,54.347826,False,False,True,True,24.050633
6,ACURA,ILX,2014,85.0,85.0,14.285714,3.5,20.0,77.0,2,80.281690,80.281690,50.000000,52.173913,False,False,True,True,25.316456
7,ACURA,ILX,2013,85.0,85.0,14.285714,3.5,20.0,81.0,2,87.323944,87.323944,58.974359,50.000000,False,False,True,True,25.316456
8,ACURA,RLX,2019,82.5,82.5,10.317460,4.0,40.0,NaN,4,NaN,-35.211268,NaN,NaN,False,False,True,True,31.645570
9,ACURA,RLX,2018,82.5,82.5,10.317460,4.0,100.0,NaN,10,NaN,-35.211268,NaN,NaN,False,False,True,True,31.645570


In [6]:
# Save the file to disk
df_car_stats.to_csv("scaled_car_stats.csv")

##### Get listing data and clean

In [8]:
df_listings = pd.read_csv("allListings.csv")
df_listings.head()

C:\Users\MyPC\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (7,12,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,year,make,model,transmission,color,drivetrain,fuel,mileage,clean_title,url,image_url,source,trim,bodystyles,listing_url,zip,lat,long
0,2014,Ford,Focus,automatic,NaN,fwd,gas,56537.0,clean,https://abilene.craigslist.org/ctd/d/comanche-...,https://images.craigslist.org/00N0N_aO010JYDnn...,craigslist,NaN,Sedan,NaN,NaN,38.94112,-97.21973
1,2015,Jeep,Renegade,automatic,white,fwd,gas,51805.0,clean,https://abilene.craigslist.org/ctd/d/comanche-...,https://images.craigslist.org/00j0j_bwjjcCXg7a...,craigslist,NaN,SUV,NaN,NaN,38.94112,-97.21973
2,2015,Ram,3500 Regular Cab,automatic,white,4wd,diesel,142679.0,clean,https://abilene.craigslist.org/ctd/d/comanche-...,https://images.craigslist.org/00M0M_3Zos6BBf8J...,craigslist,NaN,Pickup,NaN,NaN,38.94112,-97.21973
3,2015,Toyota,Highlander,automatic,white,fwd,gas,89203.0,clean,https://abilene.craigslist.org/ctd/d/comanche-...,https://images.craigslist.org/00707_5gQe8RhbBs...,craigslist,NaN,SUV,NaN,NaN,38.94112,-97.21973
4,2012,Chevrolet,Tahoe,automatic,white,4wd,gas,107932.0,clean,https://abilene.craigslist.org/ctd/d/comanche-...,https://images.craigslist.org/00e0e_ZeaNqexiJo...,craigslist,NaN,SUV,NaN,NaN,38.94112,-97.21973


In [ ]:
# Drop listings with duplicate information of (make, year, model, mileage, lat, long):
print('before removing duplicates:', len(df_listings))
df_listings_drop_fields = df_listings[['make','model','year','mileage','lat','long']]

duplicate_indices = df_listings_drop_fields[df_listings_drop_fields.duplicated()].index.tolist()
df_listings = df_listings.drop(duplicate_indices)
print('after removing duplicates:', len(df_listings))

In [ ]:
# Rename columns:
df_listings = df_listings.rename(columns={"year": "Year", "make": "Make", "model": "Model"})

# Upper case all the makes and models
df_listings['Make'] = df_listings['Make'].str.upper()
df_listings['Model'] = df_listings['Model'].str.upper()


##### Join car stats data and listing data

In [ ]:
# Add car_stats to listing dataframe using left joint
from functools import reduce

#data_frames = [small_test, df_car_stats]
data_frames = [df_listings, df_car_stats]

df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['Make', 'Model', 'Year'],
                                            how='left'), data_frames)
df_merged.head()

In [ ]:
# Drop listings with at least 1 NaN value in the following 6 fields: 
# JD Reliability, KBB Consumer Rating, JD Driving Experience, Aggregate Rating, Resale Value, KBB Combined Fuel Economy

# Remove rows that do not have (make,model,year) from the standards
drop_variable_names = ["JD Quality and Reliability", "KBB Cons Rating", "JD Driving Experience", "Aggregate Rating", "JD Resale", "KBB Combined Fuel Economy"]

for name in drop_variable_names:
    print('before remove nan values of ', name, len(df_merged))
    
    # Get names of indexes for which column Age has value ''
    indexNames = df_merged[np.isnan(df_merged[name])].index
    # Delete these row indexes from dataFrame
    df_merged.drop(indexNames , inplace=True)

    print('after remove null values of ', name,  len(df_merged))



In [ ]:
# Create a new column called scrappage rate
scrappage = [0, 0, 0.87, 0.92, 1.30, 1.62, 2.11, 2.56, 3.38, 4.24, 5.59, 6.96, 8.84, 10.64, 12.78]
scrappage = [i/100 for i in scrappage]
vehicle_age = [i for i in range(14)]
vehicle_scrappage = dict(zip(vehicle_age, scrappage))
#average life time is 15.59, with a 95% CI of [14.93, 15.56]

#https://faculty.sites.uci.edu/kevinroth/files/2011/03/Scrappage_18Jan2016.pdf
def getScrappage(row):
    year = float(row["Year"])
    age = 2020 - year
    if age <= 13:
        return vehicle_scrappage[age]
    else:
        return vehicle_scrappage[13]

#1 - vehicle_scrappage[5]
df_merged["scrappage_rate"] = df_merged.apply(lambda row: getScrappage(row), axis = 1)

In [ ]:
df_merged["Reliability"] = 0.4*(1-df_merged["scrappage_rate"])*df_merged["JD Quality and Reliability"]+0.4*df_merged["Recall Score"]+0.2*df_merged["KBB Cons Rating"]
df_merged["Driving Experience"] = 0.5*df_merged["JD Driving Experience"]+0.5*df_merged["Aggregate Rating"]
df_merged["Resale Value"] = df_merged["JD Resale"]
df_merged["Fuel Economy"] = df_merged["KBB Combined Fuel Economy"]
correction_mpg = 0.5
df_merged["Total Score"] = 0.25*df_merged["Reliability"]+0.25*df_merged["Driving Experience"]+0.25*df_merged["Resale Value"]+0.25*df_merged["Fuel Economy"]*correction_mpg
df_merged.head()

In [ ]:
# Save file to disk
df_merged.to_csv('ranked_listings_02.csv', index=False)

## Clean Zipcode data

In [ ]:
df_zipcode = pd.read_csv("us-zip-code-latitude-and-longitude.csv", delimiter = ';')

In [ ]:
df_zipcode.head()

In [ ]:
#Drop column 'geopoint'
df_zipcode = df_zipcode.drop(['geopoint'], axis = 1)
df_zipcode.head()


In [ ]:
#Save to disk
df_zipcode.to_csv('cleaned_zipcode.csv')